In [1]:
%matplotlib inline
import pyfolio as pf

In [2]:
!zipline bundles

quandl <no ingestions>
quantopian-quandl 2017-09-27 10:45:08.190035


In [3]:
import zipline
from zipline.data import benchmarks
from zipline.api import order_target, record, symbol
from zipline.utils.run_algo import run_algorithm
from datetime import datetime
import pytz

In [4]:
#%load_ext zipline

In [5]:
#%%zipline --start 2010-1-1 --end 2016-1-1
#from zipline.api import order_target, record, symbol

small_window = 3
big_window = 5

def initialize(context):
    context.i = 0
    context.asset = symbol('AAPL')


def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < big_window:
        return
    # Compute averages
    # data.history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = data.history(context.asset, 'price', bar_count=small_window, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=big_window, frequency="1d").mean()

    # Trading logic
    if short_mavg > long_mavg:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(context.asset, 20)
    elif short_mavg < long_mavg:
        order_target(context.asset, -20)

In [6]:
results = run_algorithm(datetime(2010,1,1, tzinfo=pytz.utc), 
                        datetime(2017,1,1, tzinfo=pytz.utc),
                        initialize,
                        10000,
                        handle_data=handle_data)

KeyError: 'the label [2010-01-04 00:00:00+00:00] is not in the [index]'

In [ ]:
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(results)

In [ ]:
returns[-5:]

In [ ]:
positions.head()

In [ ]:
transactions.tail()

In [ ]:
pf.create_full_tear_sheet(returns, positions, transactions)